<a href="https://colab.research.google.com/github/srivatsan88/Natural-Language-Processing/blob/master/Text_classification_Tensorflow_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to video explaining the code is availale at - https://youtu.be/dkpS2g4K08s

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
import tensorflow_hub as hub
import numpy as np
    
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
tf.test.is_built_with_cuda()

True

In [2]:
#df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')
df=pd.read_excel('USCANCom.xlsx')

In [3]:
df.columns

Index(['Company Name', 'Primary Sector', 'Primary Industry',
       'Business Description'],
      dtype='object')

In [4]:
df['Business Description'] = df['Business Description'].str.replace('[^\w\s]','')

In [5]:
pd.set_option('display.max_colwidth', -1)
df.head(1)


<ipython-input-5-176f38559c2f>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Company Name,Primary Sector,Primary Industry,Business Description
0,@Plan.Inc,Industrials,Research and Consulting Services,PlanInc was acquired by DoubleClick Inc on February 2 2001 PlanInc provided target market research planning systems for Internet advertisers advertising agencies Web publishers online retailers and consumer brand marketers


In [6]:
import string
from nltk.corpus import stopwords
import re
# # In case of any corpus are missing 
# download all-nltk


df['Business Description'] = df['Business Description'].str.replace('\d+', '')

In [7]:
df['Primary Industry'] = df['Primary Industry'].str.replace('\d+', '')

In [8]:
df['Business Description'] = df['Business Description'].astype(str)
df['Primary Sector'] = df['Primary Sector'].astype(str)

In [9]:
df.fillna(value='', inplace=True)

In [10]:
df.head(1)

,Company Name,Primary Sector,Primary Industry,Business Description
0,@Plan.Inc,Industrials,Research and Consulting Services,PlanInc was acquired by DoubleClick Inc on February PlanInc provided target market research planning systems for Internet advertisers advertising agencies Web publishers online retailers and consumer brand marketers


In [11]:
df = df[['Primary Sector','Business Description']]

In [15]:
df = df[df['Primary Sector'].isin(['Financials','Materials','Information Technology', 'Health Care','Industrials','Consumer Discretionary', 'Energy','Communication Services'])]

In [16]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=111)

In [17]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(df['Primary Sector']),
                                             df['Primary Sector']))

C:\Users\jharu\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass classes=['Communication Services' 'Consumer Discretionary' 'Energy' 'Financials'
 'Health Care' 'Industrials' 'Information Technology' 'Materials'], y=0        Industrials           
1        Information Technology
2        Consumer Discretionary
3        Consumer Discretionary
4        Financials            
            ...                
22611    Communication Services
22612    Materials             
22613    Materials             
22614    Information Technology
22615    Consumer Discretionary
Name: Primary Sector, Length: 21205, dtype: object as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [18]:
df['Primary Sector'].value_counts()

Financials                5245
Materials                 2906
Information Technology    2749
Health Care               2682
Industrials               2219
Consumer Discretionary    2014
Energy                    1918
Communication Services    1472
Name: Primary Sector, dtype: int64

In [19]:
#df = df[df['Primary Sector'].isin(['Financials','Materials','Information Technology', 'Health Care','Industrials','Consumer Discretionary', 'Energy','Communication Services'])]

In [20]:
class_weights.sort()

In [21]:
class_weights

[0.5053622497616778,
 0.9121214728148658,
 0.9642142597308112,
 0.9883016405667412,
 1.1945132942767012,
 1.316099801390268,
 1.381973409801877,
 1.800696331521739]

In [22]:
weights={}

In [23]:
for index, weight in enumerate(class_weights) :
    
    weights[index]=weight

In [24]:
weights

{0: 0.5053622497616778,
 1: 0.9121214728148658,
 2: 0.9642142597308112,
 3: 0.9883016405667412,
 4: 1.1945132942767012,
 5: 1.316099801390268,
 6: 1.381973409801877,
 7: 1.800696331521739}

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21205 entries, 0 to 22615
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Primary Sector        21205 non-null  object
 1   Business Description  21205 non-null  object
dtypes: object(2)
memory usage: 497.0+ KB


In [26]:
df[df['Business Description'].isna()]

,Primary Sector,Business Description


In [27]:
#

In [28]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train['Business Description'].values, X_train['Primary Sector'].values))
dataset_test = tf.data.Dataset.from_tensor_slices((X_test['Business Description'].values, X_test['Primary Sector'].values))

In [29]:
for text, target in dataset_train.take(1):
    
    print ('Complaint: {}, Target: {}'.format(text, target))

Complaint: b'In March  Tehama Bancorp merged with and into Humboldt Bancorp Pursuant to the merger Tehama Bank the wholly owned subsidiary of Tehama Bancorp became a separate subsidiary of Humboldt Tehama Bancorp through Tehama Bank was engaged in the commercial banking business as of September   Its services included accepting demand savings and time deposits issuing letters of credit and making commercial real estate and consumer loans The bank also offered installment note collection cashiers checks and travelers checks as well as operated as licensed merchant bankcard sales clearer Besides it provided hour automated teller service bankbymail and night depository services safe deposit boxes and other customary banking services The bank primarily served Tehama Butte Glenn and Shasta Counties California through its six offices', Target: b'Financials'


In [33]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(['Financials','Materials','Information Technology', 'Health Care','Industrials','Consumer Discretionary', 'Energy','Communication Services']),
        values=tf.constant([0, 1, 2, 3, 4, 5,6,7]),
    ),
    default_value=tf.constant(-1),
    name="target_encoding"
)

@tf.function
def target(x):
    return table.lookup(x)

In [34]:
def show_batch(dataset, size=5):
    
    
    for batch, label in dataset.take(size):
        
        print(batch.numpy())
        print(target(label).numpy())

In [35]:
show_batch(dataset_test,1)

b'DIRTT Environmental Solutions Ltd designs manufactures and installs prefabricated interior solutions for use primarily in commercial spaces across various industries and businesses in the United States Canada internationally Its ICE software interior construction technology provides integration and management from design through engineering manufacturing and installation The companys interior construction solutions include prefabricated customized interior modular walls ceilings and floors decorative and functional millwork power infrastructure network infrastructure and preinstalled medical gas piping systems It offers interior construction solutions through a network of independent distribution partners It serves commercial healthcare education hospitality and other industries as well as medical gas piping systems for healthcare DIRTT Environmental Solutions Ltd was founded in  and is headquartered in Calgary Canada'
4


In [36]:
def fetch(text, labels):
    
    return text, tf.one_hot(target(labels),8)

In [37]:
train_data_f=dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [38]:
next(iter(train_data_f))

(<tf.Tensor: shape=(), dtype=string, numpy=b'In March  Tehama Bancorp merged with and into Humboldt Bancorp Pursuant to the merger Tehama Bank the wholly owned subsidiary of Tehama Bancorp became a separate subsidiary of Humboldt Tehama Bancorp through Tehama Bank was engaged in the commercial banking business as of September   Its services included accepting demand savings and time deposits issuing letters of credit and making commercial real estate and consumer loans The bank also offered installment note collection cashiers checks and travelers checks as well as operated as licensed merchant bankcard sales clearer Besides it provided hour automated teller service bankbymail and night depository services safe deposit boxes and other customary banking services The bank primarily served Tehama Butte Glenn and Shasta Counties California through its six offices'>,
 <tf.Tensor: shape=(8,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

In [39]:
train_data, train_labels = next(iter(train_data_f.batch(5)))
train_data, train_labels

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'In March  Tehama Bancorp merged with and into Humboldt Bancorp Pursuant to the merger Tehama Bank the wholly owned subsidiary of Tehama Bancorp became a separate subsidiary of Humboldt Tehama Bancorp through Tehama Bank was engaged in the commercial banking business as of September   Its services included accepting demand savings and time deposits issuing letters of credit and making commercial real estate and consumer loans The bank also offered installment note collection cashiers checks and travelers checks as well as operated as licensed merchant bankcard sales clearer Besides it provided hour automated teller service bankbymail and night depository services safe deposit boxes and other customary banking services The bank primarily served Tehama Butte Glenn and Shasta Counties California through its six offices',
        b'As of July   Mission Resources Corp was acquired by Petrohawk Energy Corp Mission Resources Corporation e

In [40]:
train_data[:1]

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'In March  Tehama Bancorp merged with and into Humboldt Bancorp Pursuant to the merger Tehama Bank the wholly owned subsidiary of Tehama Bancorp became a separate subsidiary of Humboldt Tehama Bancorp through Tehama Bank was engaged in the commercial banking business as of September   Its services included accepting demand savings and time deposits issuing letters of credit and making commercial real estate and consumer loans The bank also offered installment note collection cashiers checks and travelers checks as well as operated as licensed merchant bankcard sales clearer Besides it provided hour automated teller service bankbymail and night depository services safe deposit boxes and other customary banking services The bank primarily served Tehama Butte Glenn and Shasta Counties California through its six offices'],
      dtype=object)>

In [41]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 0.9678121 ,  0.21130985,  0.4878323 , -0.30263278, -0.29055503,
         0.26833868,  0.44282416,  0.07498636, -0.26262996,  0.19421248,
         0.13042702,  0.10800909, -0.29733106,  0.13081373,  0.12934518,
         0.05613798, -0.3534584 ,  0.30905288, -0.02757133,  0.8682679 ,
        -0.364638  ,  0.21522994, -0.02652664,  0.2800543 ,  0.06661361,
        -0.4496872 ,  0.09921843, -0.00197166, -0.05644349,  0.22302598,
         0.1345886 , -0.28414991, -0.35626101,  0.11043958, -0.2343015 ,
         0.22904566, -0.07671241, -0.10843585, -0.26986772, -0.08354723,
         0.25766337, -0.13431184,  0.18929198,  0.07759936,  0.07133481,
         0.14835729,  0.09576484,  0.17312695, -0.11157478,  0.10907102,
        -0.00582284, -0.3489654 ,  0.53033006,  0.17980231, -0.28084248,
        -0.18559681, -0.12318989,  0.17547438, -0.04151918, -0.17721838,
         0.25698146,  0.24676938,  0.00259879, -0.26612645, -0.0890199 ,
 

In [42]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Reshape( target_shape=(128 , 1 ) ))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
for units in [128, 64 ]:
    
    model.add(tf.keras.layers.Dense(units, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(8, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
reshape (Reshape)            (None, 128, 1)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 128)          33792     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8

In [43]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [44]:
train_data_f=train_data_f.shuffle(70000).batch(100)
test_data_f=test_data_f.batch(100)

In [45]:
history = model.fit(train_data_f,
                    epochs=4,
                    validation_data=test_data_f,
                    verbose=1,
                    class_weight=weights)

Epoch 1/4
170/170 [==============================] - 430s 3s/step - loss: 1.8434 - accuracy: 0.5070 - val_loss: 1.6025 - val_accuracy: 0.6682
Epoch 2/4
170/170 [==============================] - 371s 2s/step - loss: 1.5940 - accuracy: 0.7174 - val_loss: 1.5363 - val_accuracy: 0.7369
Epoch 3/4
170/170 [==============================] - 368s 2s/step - loss: 1.5035 - accuracy: 0.7947 - val_loss: 1.5336 - val_accuracy: 0.7380
Epoch 4/4
170/170 [==============================] - 372s 2s/step - loss: 1.4702 - accuracy: 0.8209 - val_loss: 1.5307 - val_accuracy: 0.7399


In [46]:
len(list(dataset_test))

4241

In [47]:
results = model.evaluate(dataset_test.map(fetch).batch(11491), verbose=2)

print(results)

1/1 - 0s - loss: 1.5307 - accuracy: 0.7399
[1.530747890472412, 0.7399198412895203]


In [48]:
test_data, test_labels = next(iter(dataset_test.map(fetch).batch(45963)))

In [49]:
y_pred=model.predict(test_data)

In [50]:
from sklearn.metrics import classification_report

In [51]:
print(classification_report(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85      1079
           1       0.92      0.77      0.84       621
           2       0.80      0.49      0.61       558
           3       0.92      0.83      0.87       527
           4       0.48      0.65      0.55       427
           5       0.54      0.69      0.61       401
           6       0.82      0.89      0.85       362
           7       0.45      0.68      0.54       266

    accuracy                           0.74      4241
   macro avg       0.72      0.73      0.72      4241
weighted avg       0.77      0.74      0.75      4241



In [52]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1))

array([[890,  15,   9,   6,  48,  35,   9,  67],
       [ 24, 481,   2,   3,  70,  12,  24,   5],
       [ 34,   4, 273,   9,  93,  58,   4,  83],
       [  6,   1,  10, 437,  18,  32,   9,  14],
       [ 20,   7,  29,  12, 278,  51,  15,  15],
       [ 25,   2,   7,   5,  46, 276,   6,  34],
       [  3,  13,   0,   2,  17,   3, 323,   1],
       [ 11,   2,  12,   2,  11,  43,   5, 180]], dtype=int64)

In [56]:
new_complaint = [' Medicines and vaccines are going to be our priority ']
#seq = tokenizer.texts_to_sequences(new_complaint)
#padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(new_complaint)
labels = ['Financials','Materials','Information Technology', 'Health Care','Industrials','Consumer Discretionary', 'Energy','Communication Services']
print(pred, labels[np.argmax(pred)])

[[4.3813248e-05 2.1499852e-03 4.9917289e-05 2.1862360e-02 8.2536530e-01
  6.3375264e-02 8.7094389e-02 5.8929851e-05]] Industrials
